In [1]:
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib notebook 
plt.rcParams["figure.figsize"] = (3,3)
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import solve_ivp

Suivi de position en mer
======================

Le but de ce projet est de simuler et prevoir en temps reel la position de navires en mer à partir de donnees GPS. En effet, la plupart de la flotte mondiale est maintenant equippee d'un AIS (Automatic Identification System) 

https://shipping.nato.int/nsc/operations/news/2021/ais-automatic-identification-system-overview

qui communique regulierement aux côtes et aux satellites la position de chaque bateau. Les donnees sont en fait disponibles publiquement et en temps reel sur 

https://www.marinetraffic.com/en/ais/home/centerx:-4.5/centery:50.8/zoom:6

Ces communications permettent le suivi des positions et l'identification des navires à la fois aux côtes et à chacun des navires pour eviter les collisions.

Considerons un bateau modelise par sa position 2D $(x_1,x_2)\in \mathbb{R}^2$ dans le plan de la mer (on neglige la houle), et son orientation $x_3\in \mathbb{R}$
\begin{align*}
\dot{x}_1 &= v(t) \cos(x_3) \\
\dot{x}_2 &= v(t) \sin(x_3) \\
\dot{x}_3 &= w(t)
\end{align*}
où $v$ et $w$ sont les vitesses lineaires et de rotation respectivement. On souhaite estimer la position en temps reel de ce bateau à partir de mesures GPS bruitees sans connaître les signaux $v$ et $w$. Pour cela, nous allons d'abord etudier la simulation numerique des trajectoires de ce bateau, puis apprendre à implementer un *filtre de Kalman* permettant le filtrage des donnees GPS grâce à la connaissance d'un modele approxime. 

Notons que dans ce projet, nous supposons la position GPS disponible continûment (à chaque instant) alors qu'elle est en fait echantillonnee en pratique, et il faut alors utiliser des methodes discretes mêlant la prediction par le modele et le recalage par les mesures (filtre de Kalman discret, filtre particulaire). Ce sont des algorithmes probabilistes que vous verrez dans le projet numerique de probabilites.


In [2]:
def f(t,x):
    v = 100
    w = 100
    return np.array([v*np.cos(x[2]),
                     v*np.sin(x[2]),
                     w])

Euler explicite
================

**Question 1** Completer les fonctions ``solve_euler_explicit`` et ``predict_euler_explicit`` ci-dessous prenant en entree une fonction $f: \mathbb{R}\times\mathbb{R}^n \to \mathbb{R}^n$ quelconque, une condition initiale $x_0$, un pas de temps $dt$, les temps initiaux et finaux, et renvoyant 

- le vecteur des temps $t^j$ et de la solution $x^j$ du schema d'Euler explicite applique à $\dot{x}=f(t,x)$,

- la valeur finale au temps $t_f$ de la solution du schema d'Euler explicite applique à $\dot{x}=f(t,x)$,

respectivement.

In [3]:
def solve_euler_explicit(f, x0, dt, t0, tf):
    t, x = [t0], [x0]
    while t[-1]<= tf:
        x_iter = x[-1] + dt*f(t[-1], x[-1])
        x.append(x_iter)
        t.append(t[-1] + dt)
    
    return np.array(t),np.array(x)

In [4]:
def predict_euler_explicit(f, x0, dt, t0, tf):
    t_list,x_list = solve_euler_explicit(f, x0, dt, t0, tf)
    return x_list[-1] + (tf-t_list[-1])*f(t_list[-1], x_list[-1])

**Question 2** Utiliser ``solve_euler_explicit`` pour simuler des trajectoires simples et connues de bateau (ligne droite et cercle notamment). Etudier l'erreur numerique du schema
- lorsque $\Delta t$ tend vers 0 pour un horizon de temps $t_f$ fixe ; 
- à $\Delta t$ fixe, lorsque $t_f$ augmente : arrivez-vous à simuler une trajectoire circulaire en temps long?

Expliquer comment trouver numeriquement (sans le faire) l'ordre de convergence du schema.

In [5]:
def f_ligne(t,x):
    """modelise une trajectoire rectiligne"""
    v = 100
    w = 0
    return np.array([v*np.cos(x[2]),
                     v*np.sin(x[2]),
                     w])

x0 = np.array([100,100,20])
t_list, x_list = solve_euler_explicit(f_ligne, x0, 1, 0, 10)

plt.figure()
plt.plot(x_list[:,0],x_list[:,1])
plt.grid(True)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')

<IPython.core.display.Javascript object>

Text(0, 0.5, '$x_2$')

In [6]:
# erreur numerique dans le cas de la ligne droite
v1 = 100

def norme(X):
    return np.sqrt(X[0]**2 + X[1]**2 + X[2]**2)
# vec_norme = np.vectorize(norme) 

vec_norme = np.vectorize(norme)

def x1(t):
    return v1*np.cos(x0[2])*t + x0[0]

vec_x1 = np.vectorize(x1)

def x2(t):
    return v1*np.sin(x0[2])*t + x0[1]

vec_x2 = np.vectorize(x2)

def erreur_numerique1(dt):
    Z = solve_euler_explicit(f_ligne, x0, dt, 0, 10)
    X_approche = Z[1]
    X_reel = np.transpose([vec_x1(Z[0]), vec_x2(Z[0]), np.full(len(Z[0]),x0[2])])
    erreur = [norme(X_approche[i] - X_reel[i]) for i in range(len(Z[0]))]
    # erreur = vec_norme(X_approche - X_reel)
    return max(erreur)
  
vec_erreur_numerique1 = np.vectorize(erreur_numerique1)

list_t = np.linspace(0.01,10,50)

plt.figure()
plt.plot(list_t, vec_erreur_numerique1(list_t))
plt.xlabel('pas de temps')
plt.ylabel('erreur numerique')
plt.grid(True)
plt.show()

<IPython.core.display.Javascript object>

In [7]:
# erreur numerique dans le cas de la ligne droite à dt fixe
dt = 1

def erreur_numerique2(tf):
    Z = solve_euler_explicit(f_ligne, x0, dt, 0, tf)
    X_approche = Z[1]
    X_reel = np.transpose([vec_x1(Z[0]), vec_x2(Z[0]), np.full(len(Z[0]),x0[2])])
    erreur = [norme(X_approche[i] - X_reel[i]) for i in range(len(Z[0]))]
    return max(erreur)
  
vec_erreur_numerique2 = np.vectorize(erreur_numerique2)

plt.figure()
list_tf = np.linspace(0, 40, 500)
plt.plot(list_tf, vec_erreur_numerique2(list_tf))
plt.xlabel('temps final')
plt.ylabel('erreur numerique')
plt.grid(True)
plt.show()

<IPython.core.display.Javascript object>

In [8]:
def f_cercle(t,x):
    """modelise une trajectoire circulaire"""
    v = 100
    w = 100
    return np.array([v*np.cos(x[2]),
                     v*np.sin(x[2]),
                     w])

x0 = np.array([100,100,0])
t_list, x_list = solve_euler_explicit(f_cercle, x0, 1, 0, 200)

plt.figure()
plt.plot(x_list[:,0],x_list[:,1])
plt.grid(True)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')



<IPython.core.display.Javascript object>

Text(0, 0.5, '$x_2$')

In [9]:
# erreur numerique dans le cas du cercle
v=100
w=100


def x1(t):
    return v*(np.sin(w*t + x0[2]) - np.sin(x0[2]))/w + x0[0]

vec_x1 = np.vectorize(x1)

def x2(t):
    return v*(np.cos(x0[2]) - np.cos(w*t + x0[2]))/w + x0[1]

vec_x2 = np.vectorize(x2)

def W(t):
    return w*t + x0[2]

vec_W = np.vectorize(W)

def erreur_numerique2(dt):
    Z = solve_euler_explicit(f_cercle, x0, dt, 0, 10)
    X_approche = Z[1]
    X_reel = np.transpose([vec_x1(Z[0]), vec_x2(Z[0]), vec_W(Z[0])])
    erreur = [norme(X_approche[i] - X_reel[i]) for i in range(len(Z[0]))]
    return max(erreur)
  
vec_erreur_numerique2 = np.vectorize(erreur_numerique2)

plt.figure()
list_t = np.linspace(0.01,10, 50)
plt.plot(list_t, vec_erreur_numerique2(list_t))
plt.xlabel('pas de temps')
plt.ylabel('erreur numerique')
plt.grid(True)
plt.show()

<IPython.core.display.Javascript object>

In [10]:
# erreur numérique dans le cas du cercle à dt fixé
dt = 1

def erreur_numérique2(tf):
    Z = solve_euler_explicit(f_cercle, x0, dt, 0, tf)
    X_approché = Z[1]
    X_réel = np.transpose([vec_x1(Z[0]), vec_x2(Z[0]), vec_W(Z[0])])
    erreur = [norme(X_approché[i] - X_réel[i]) for i in range(len(Z[0]))]
    return max(erreur)
  
vec_erreur_numérique2 = np.vectorize(erreur_numérique2)

plt.figure()
list_tf = np.linspace(0, 40, 500)
plt.plot(list_tf, vec_erreur_numérique2(list_tf))
plt.xlabel('temps final')
plt.ylabel('erreur numérique')
plt.grid(True)
plt.show()

<IPython.core.display.Javascript object>

# Determination numerique de l'ordre de convergence

On trace pour differentes valeurs de l'entier p la fonction $ \Delta t ~ {\longmapsto} ~ \frac{erreur~numerique}{{(\Delta t)}^p} $ et on regarde graphiquement le dernier entier p pour lequel cette fonction est bornee.

Modele d'estimation
====================

Notre fonction de prediction demande un pas de temps suffisamment petit et surtout necessite de connaitre les vitesses $v$ et $w$ du bateau. On souhaite être capable de suivre la trajectoire du bateau sans connaitre ces quantites, en exploitant des mesures de position intermittentes obtenues par GPS. On suppose pour cela que les vitesses $v$ et $w$ varient lentement et on adopte donc le modele etendu (approxime)
\begin{align}
\dot{x}_1 &= x_4 \cos(x_3) \\
\dot{x}_2 &= x_4 \sin(x_3) \\
\dot{x}_3 &= x_5 \\
\dot{x}_4 &= 0 \\
\dot{x}_5 &= 0 
\end{align}

In [11]:
def f_ext(t,x):
    return np.array([x[3]*np.cos(x[2]),x[3]*np.sin(x[2]),x[4],0,0])

On suppose dans ce projet que l'on dispose d'une mesure GPS de position $y=(x_1,x_2)$ à chaque instant et que l'on souhaite estimer la position $(x_1,x_2)$, l'angle $x_3$ et les vitesses $(x_4,x_5)$ en temps reel. Estimer la position dejà connue par le GPS peut paraître inutile mais la mesure est souvent bruitee et l'information du modele d'evolution peut permettre de reduire ou "filtrer" ce bruit.

**Question 3** Justifier que le systeme theorique donne par ``f_ext`` et $y$ est *observable*, \eta'est-à-dire que la connaissance de $t\mapsto y(t) = (x_1(t),x_2(t))$ determine de maniere unique tout l'etat $(x_1,x_2,x_3,x_4,x_5)$. 

On peut determiner respectivement $x_4$, $x_3$ et $x_5$ par les formules suivantes :

$ x_4 = \sqrt{\dot{x_1}^2 + \dot{x_2}^2} \\
 x_3 = \arctan(\frac{\dot{x_2}}{\dot{x_1}}) \\
 x_5 = \dot{x_3} $



On souhaite donc ecrire un algorithme, appele *observateur* ou *filtre*, qui prenne en entree $y(t)$ et qui construise une estimee $\hat{x}(t)$ de $x(t)$ telle que $\hat{x}$ converge vers $x$ asymptotiquement.

Pour cela, il faut prendre en compte que le modele donne par ``f_ext`` et $y$ n'est pas exact car il y a 2 sources d'incertitudes : les erreurs de mesure du GPS, et les accelerations $\dot{v}$ et $\dot{w}$ qui ne sont pas nulles en realite. Le systeme reel est donc plutôt
\begin{align*}
\dot{x}_1 &= x_4 \cos(x_3) \\
\dot{x}_2 &= x_4 \sin(x_3) \\
\dot{x}_3 &= x_5 \hspace{5em} , \hspace{5em} y_m = (x_1 ,x_2) + \delta_y(t) \\
\dot{x}_4 &= \delta_v(t) \\
\dot{x}_5 &= \delta_w(t)
\end{align*}
avec $\delta_v\in \mathbb{R}$, $\delta_w\in \mathbb{R}$, $\delta_y\in \mathbb{R}^2$, des entrees *faibles* mais inconnues. M'observateur doit donc être *robuste* aux erreurs de modeles et de mesure.

Etude theorique du filtre de Kalman etendu deterministe
==============

Considerons un systeme dynamique decrit par
$$
\dot{x} = f(t,x) + \delta_f(t) \quad , \quad y_m = h(t,x)+ \delta_h(t) 
$$
où $x\in \mathbb{R}^n$ est l'etat du systeme, $y_m\in \mathbb{R}^p$ est la mesure connue, et $\delta_f,\delta_h$ sont des incertitudes inconnues que l'on suppose bornees. 

Etant donne la mesure $t\mapsto y_m(t)$, on s'interesse alors à implementer un *filtre de Kalman* donne par
\begin{align}
\dot{\hat{x}} & = f(t,\hat{x})+ P\frac{\partial h}{\partial x}(t,\hat{x})^\top    R^{-1} (y_m(t)-h(t,\hat{x})) 
\\
\dot{P} & = \lambda P + \frac{\partial f}{\partial x}(t,\hat{x}) P + P\frac{\partial f}{\partial x}(t,\hat{x})^\top    + Q - P \frac{\partial h}{\partial x}(t,\hat{x})^\top    R^{-1} \frac{\partial h}{\partial x}(t,\hat{x}) P 
\end{align}
où $\hat{x}\in \mathbb{R}^n$, $P\in \mathbb{R}^{n\times n}$ constitue l'etat du filtre, et où les parametres $\lambda\in \mathbb{R}$, $Q\in \mathbb{R}^{n\times n}$, $R\in \mathbb{R}^{p}\times P$ sont à choisir tels que
- $\lambda \geq 0$ et $Q$ est symetrique, à valeurs propres positives (matrice positive),
- $R$ est symetrique, à valeurs propres strictement positives (matrice definie positive),
- $\lambda > 0$ ou bien $Q$ est à valeurs propres strictement positives.

Cet algorithme compare à chaque instant les mesures aux estimees et corrige ainsi ces estimees dynamiquement, en temps reel. En effet, $x(0)$ etant mal connue, le modele etant imparfait, et les erreurs numeriques s'accumulant, la simulation sans ces corrections serait impossible. On va donc montrer que modulo les erreurs de modele et de mesure, \eta'est algorithme converge localement, \eta'est-à-dire, $\hat{x}$ tend vers $x$ si l'erreur initiale $\hat{x}(0)-x(0)$ est suffisamment faible.

Pour cela, on admet ici que 

- si $P(0)$ est symetrique à valeurs propres strictement positives, alors $P(t)$ le reste pour tout $t\geq 0$. 
- De plus, on admet que le long des trajectoires, il existe $\rho_1>0$ et $\rho_2>0$ tels que 
$$
\rho_1 I_n \leq P^{-1}(t) \leq \rho_2 I_n \quad \forall t\geq 0 \, (0.5)
$$
où $I_n$ est la matrice identite de dimension $n$. En particulier, $P$ ne tend pas à devenir singuliere, ni diverger. Cette derniere hypothese est cruciale dans la theorie du filtre de Kalman et se justifie sous des hypotheses d'*observabilite* dans le cas des systemes lineaires. Mais l'etude de cette hypothese nous amenerait bien trop loin pour ce projet... (voir remarque ci-dessous)

En considerant le systeme complet d'etat $(x,\hat{x},P)$ avec $P$ inversible, posons
$$
V(x,\hat{x},P) = (\hat{x}-x)^\top    P^{-1} (\hat{x}-x) \ .
$$
et 
$$
g(t) = V(x(t),\hat{x}(t),P(t))
$$

**Question 4** On suppose que les derivees partielles de $f$ et $h$ par rapport à $x$ au premier et second ordre sont globalement bornees. $(1)$ 

En deduire qu'etant donnes $Q,R,\lambda, \rho_1,\rho_2$, 

il existe $\kappa_f\geq 0$, $\kappa_h\geq 0$, $k>0$, et $\varepsilon >0$, $\delta\geq 0$, tels que 

pour tout $(t,x,\hat{x})\in \mathbb{R}^+ \times \mathbb{R}^n \times \mathbb{R}^n$,
\begin{align}
\left\|f(t,x)-f(t,\hat{x})-\frac{\partial f}{\partial x}(t,\hat{x}) (x-\hat{x})\right\| &\leq \kappa_f \|x-\hat{x}\|^2 \, (2)\\
\left\|h(t,x)-h(t,\hat{x})-\frac{\partial h}{\partial x}(t,\hat{x}) (x-\hat{x})\right\| &\leq \kappa_h \|x-\hat{x}\|^2 \, (3)
\end{align}
et 
$$
\frac{d}{dt}V(x(t),\hat{x}(t),P(t)) \leq - k V(x(t),\hat{x}(t),P(t)) + \delta \, (4)
$$
le long des trajectoires verifiant $\|\hat{x}(t)-x(t) \|<\varepsilon$. 

En deduire qu'il existe $k'>0,\rho'>0,\delta'\geq 0$ tels que 

si $\|\hat{x}(0)-x(0)\|$, $\delta_f$ et $\delta_h$ sont suffisamment faibles, 

alors
$$
\|\hat{x}(t)-x(t) \| \leq \rho' e^{-k' t} \|\hat{x}(0)-x(0) \|  + \delta' \qquad \forall t\geq 0, \, (5)
$$
avec $\delta'=0$ en l'absence d'incertitudes, i.e., si $\delta_f=0$ et $\delta_h=0$.

*Indice* : 
- Montrer que $\frac{d}{dt}P^{-1}(t)=-P^{-1}(t) \left(\frac{d}{dt}P(t) \right) P^{-1}(t) \, (6) $. 

- On rappelle aussi que par Cauchy-Schwarz, pour tout $(x,y)\in \mathbb{R}^n\times \mathbb{R}^n$ et pour toute matrice symetrique $P$ à valeurs propres strictement positives (matrice definie positive), on a $x^\top    P y \leq \sqrt{x^\top    P x} \sqrt{y^\top    P y}$. 

- On rappelle aussi l'inegalite de Young :  pour tout $(a,b) \in \mathbb{R}\times \mathbb{R}$ et pour tout $\gamma>0$, $2 ab \leq \gamma a^2 +\frac{1}{\gamma} b^2$.

**Preliminaires**

Notons $||\cdot||$ la norme infinie sur $\mathbb{R}^n$ (avec la base canonique $(e_i)_i$), $||\cdot||_{\infty}$ celle sur l'espace des fonctions bornées de $\mathbb{R}^n$ dans $\mathbb{R}^n$ et $|||\cdot|||$ la norme subordonnee à la premiere ($|||A||| \hat{=} sup_{||x||=1} \|A\cdot x\|$, verifiant $|||AB||| \leq |||A||| \, |||B|||$).

$f$ est deux fois continûment differentiable.

par (1), il existe $M$ tel que pour $t \geq 0$, pour $1 \leq i,j \leq n$,  
$$\left\|\frac{\partial f}{\partial x_i}(t,\cdot)\right\|_{\infty} \leq M \\
\left\|\frac{\partial ^2 f}{\partial x_ix_j}(t,\cdot)\right\|_{\infty} \leq M$$

Notons $f_i$ la ième fonction composante de $f$ selon la base canonique de $\mathbb{R}^n$ aussi deux fois continûment differentiable.

Pour $ t,x \in \mathbb{R} \times \mathbb{R}^n$, en raisonnant sur les coordonnees du gradient et de la hessienne de $f_i$ en base canonique de $\mathbb{R}^n$ en $(t,x)$ on a alors l'existence de leurs normes subordonnees, qui sont $\leq M$ (\eta'est aussi vrai pour les differentielles partielles et les transposees de ces applications).

En effet, on detaille le cas où $f$ est à valeur reelle :

on a $\forall t,x,i \in \mathbb{R} \times \mathbb{R}^n \times \{1,...,n\}, \left| \frac{\partial f}{\partial x_i}(t,x) \right| \leq M$

donc pour $y \in \mathbb{R}$ unitaire et $t,x$, 

$$\left\|\frac{\partial f}{\partial x}(t,x)\cdot y\right\| 
= \left\|\sum_i \frac{\partial f}{\partial x_i}(t,x)y_ie_i\right\| \\
= \left|\frac{\partial f}{\partial x_a}(t,x) y_a\right| 
= \left|\frac{\partial f}{\partial x_a}(t,x)\right| \left|y_a\right| 
\leq M \, 1$$

où $1 \leq a \leq n$ est l'indice du coefficient de module maximal

et donc $$\left|\left|\left|\frac{\partial f}{\partial x} \right|\right|\right| \leq M$$

On prend de même $M$ comme majorant des gradients et hessiennes de $h$, qui respecte les mêmes hypothèses que $f$.

**Preuve de (2)**

Soit $1 \leq i \leq n $, notons $H_i(t,x)$ la hessienne de $f_i$ en $t,x$

Pour $t \geq 0 \, , \, (x,\hat{x}) \in \mathbb{R}^n \times \mathbb{R}^n$ on applique Taylor-reste integral à $f_i(t,\cdot)$ (qui est $C^2$) entre $x$ et 
$\hat{x}$, on a 

$$f_i(t,x) - f_i(t,\hat{x}) - \frac{\partial f_i}{\partial x}(t,\hat{x})\cdot(x - \hat{x}) 
= \int_{0}^{1}  < H_i(t, x + (x - \hat{x}) u) \cdot(x - \hat{x}),
(x - \hat{x}) > (1-u) du $$

Or par Cauchy-Schwarz avec le produit scalaire canonique sur $\mathbb{R}^n$ : $$<A\cdot x,x> \leq \|A\cdot x\|\|x\| \leq |||A||| \, \|x\|^2$$

donc par inegalite triangulaire integrale 

$$ \left| f_i(t,x) - f_i(t,\hat{x}) - \frac{\partial f_i}{\partial x}(t,\hat{x})\cdot(x - \hat{x})\right|
\leq \int_{0}^{1}  ||| H_i(t, x + (x - \hat{x}) u)||| \|x - \hat{x}\|^2(1-u) du \\
\leq \int_{0}^{1}  M \|x - \hat{x}\|^2(1-u) du 
= \frac{M}{2} \|x - \hat{x}\|^2
$$

donc en passant à $||\cdot||$ et en prenant $\kappa_f = \frac{M}{2}$, 

$$\left\|f(t,x)-f(t,\hat{x})-\frac{\partial f}{\partial x}(t,\hat{x}) \cdot (x-\hat{x})\right\| \leq \kappa_f \|x-\hat{x}\|^2 \, (2)$$

On fait pareil pour (3)


**Preuve de l'indice**

On dérive $  P(t) P^{-1}(t) =  I_n $
$$ \dot{ P}(t) P^{-1}(t) +  P(t)\dot{ P^{-1}}(t) = 0 \\
 \dot{ P^{-1}}(t) = - P^{-1}(t)\dot{ P}(t) P^{-1}(t) $$

**Preuve de (4)**

On rappelle $$ g(t) = V(x(t),\hat{x}(t),P(t)) $$

Notons $q(t) = \hat{x}(t)-x(t)$

$$g = q\top   P^{-1}q = <q,P^{-1}q>$$

Par bilinearite du produit matriciel  
$$\dot{g} = \dot{q}^{T}P^{-1}q + q^{T}(\dot{P^{-1}})q  + q^{T}P^{-1}\dot{q}$$

soit $k>0$, par Cauchy-Schwarz de l'indice sachant $P^{-1} > 0$

$$\dot{g} + kg \leq 2\sqrt{\dot{q}^{T}P^{-1}\dot{q}}\sqrt{q^{T}P^{-1}q} + q^{T}(\dot{P^{-1}})q + kq^{T}P^{-1}q$$

soit $\gamma>0$, par Young

$$\dot{g} + kg  \leq \gamma\dot{q}^{T}P^{-1}\dot{q} + \frac{1}{\gamma}q^{T}P^{-1}q + q^{T}\dot{P^{-1}}q + kq^{T}P^{-1}q$$

Si $A \geq 0$ alors $<Ax,x> \geq 0$ or $\rho_1I_n \leq P^{-1} \leq \rho_2I_n$ donc $\rho_1||q||^2 \leq q^{T}P^{-1}q \leq \rho_2||q||^2$ donc 

$$\dot{g} + kg  \leq \gamma \rho_2||\dot{q}||^2 + q^{T}(\dot{P^{-1}})q + \left(\frac{1}{\gamma} + k \right)\rho_2||q||^2 \, (E)$$

or

$$\dot{q} = \dot{\hat{x}}-\dot{x} = \\
f(t,\hat{x}) - f(t,x) + P\frac{\partial h}{\partial x}(t,\hat{x})^\top    R^{-1} (h(t,x) -h(t,\hat{x}) + \delta_h(t) ) - \delta_f(t) $$

$$\dot{P} = \lambda P + \frac{\partial f}{\partial x}(t,\hat{x}) P + P\frac{\partial f}{\partial x}(t,\hat{x})^\top    + Q - P \frac{\partial h}{\partial x}(t,\hat{x})^\top    R^{-1} \frac{\partial h}{\partial x}(t,\hat{x}) P$$

$$\dot{P^{-1}} = -P^{-1}(t)(\dot{P})P^{-1}(t) \\  
= - \left( \lambda P^{-1} + P^{-1}\frac{\partial f}{\partial x}(t,\hat{x}) + \frac{\partial f}{\partial x}(t,\hat{x})^\top    P^{-1} + P^{-1}QP^{-1} - \frac{\partial h}{\partial x}(t,\hat{x})^\top    R^{-1} \frac{\partial h}{\partial x}(t,\hat{x}) \right)$$

donc

$$ q^\top   \dot{P^{-1}}q = -q\top   \lambda P^{-1}q - q\top    \left(P^{-1}\frac{\partial f}{\partial x}(t,\hat{x}) + \frac{\partial f}{\partial x}(t,\hat{x})^\top    P^{-1} \right)q - q\top   P^{-1}QP^{-1}q + q\top   \frac{\partial h}{\partial x}(t,\hat{x})^\top    R^{-1} \frac{\partial h}{\partial x}(t,\hat{x})q \\
\leq - \lambda \rho_1 \|q\|^2 - 2<P^{-1}q,\frac{\partial f}{\partial x}(t,\hat{x})q> + 
\|\frac{\partial h}{\partial x}(t,\hat{x})q\| \|R^{-1} \frac{\partial h}{\partial x}(t,\hat{x})q\|
$$

car 
- $P^{-1} \geq 0$
- $Q \geq 0 $ donc $(P^{-1}q)^\top   Q(P^{-1}q) \leq 0$
- $R > 0$ donc $R^{-1} > 0$ donc en decomposant $R^{-1}x$ sur une base orthonormee de vecteurs propres, on obtient $|||R^{-1}||| \leq \mu$ où $\mu$ est la plus grande valeur propre de $R^{-1}$

donc par Cauchy-Schwarz avec $<,>$ et existence et sous multiplicite des normes subordonnees : 
$$ q\top   \dot{P^{-1}}q \leq 
- \lambda \rho_1 \|q\|^2 + 2\|P^{-1}q\|\|\frac{\partial f}{\partial x}(t,\hat{x})q\| + |||R^{-1}||| \, ||| \frac{\partial h}{\partial x}(t,\hat{x})|||^2 \, \|q\|^2 \\
\leq - \lambda \rho_1 \|q\|^2 + 2|||P^{-1}||| \, \|q\| \, |||\frac{\partial f}{\partial x}(t,\hat{x})||| \, \|q\| + |||R^{-1}||| \, M^2 \, \|q\|^2 \\
\leq (- \lambda \rho_1 + 2\rho_2 M + |||R^{-1}||| \, M^2) \|q\|^2 $$

et

$$\|\dot{q}\| 
\leq \|f(t,\hat{x}) - f(t,x)\| + \|P\frac{\partial h}{\partial x}(t,\hat{x})^\top    R^{-1} (h(t,x) -h(t,\hat{x}) + \delta_h(t) )\| + \|\delta_f(t)\| \\
\leq M\|q\| + |||P(t)|||\,|||\frac{\partial h}{\partial x}(t,\hat{x})^\top   |||\,|||R^{-1}|||\,
\|h(t,x) -h(t,\hat{x}) + \delta_h(t)\| + \|\delta_f(t)\| \\
\leq M\|q\| + \frac{1}{\rho_1}\,M\,|||R^{-1}|||\,M \left(M\|q\| + \eta\right) + \eta \\
\leq (1+\frac{1}{\rho_1}\,M\,|||R^{-1}|||\,M)M\|q\| + (1 + \frac{1}{\rho_1}\,M\,|||R^{-1}|||\,M)\eta \\
\leq N(M\|q\| + \eta)$$

avec:
- $N = (1+\frac{1}{\rho_1}\,|||R^{-1}|||\,M^2)$

car
- par le théorème des accroissements finis : $\|f(t,\hat{x}) - f(t,x)\| \leq M\|q\|$ et $\|h(t,\hat{x}) - h(t,x)\| \leq M\|q\|$
- $\delta_f(t), \, \delta_h(t)$ sont majores par $\eta$
- $P\leq \frac{1}{\rho_1} I_n$, où $\frac{1}{\rho_1} \geq 0$ est un majorant des valeurs propres de $P > 0$

Enfin $(E)$ devient :

$$\dot{g} + kg  \leq \gamma \rho_2||\dot{q}||^2 + q^{T}(\dot{P^{-1}})q + \left(\frac{1}{\gamma} + k \right)\rho_2||q||^2 \\
\dot{g}(t) + kg(t) \leq \gamma \rho_2 N^2 (M\|q\| + \eta)^2 + (- \lambda \rho_1 + 2\rho_2 M + |||R^{-1}||| \, M^2) \|q\|^2
+ \left(\frac{1}{\gamma} + k \right)\rho_2||q||^2 \\
\leq \gamma \rho_2 N^2 (M\|q\| + \eta)^2 
+ \left(- \lambda \rho_1 + 2\rho_2 M + |||R^{-1}||| \, M^2 + \left(\frac{1}{\gamma} + k \right)\rho_2 \right) ||q||^2
$$

alors soit $\varepsilon \geq 0$, on considère les trajectoires telles que $\|q(t) \| < \varepsilon$ 

alors on pose, sachant $\|q\| < \varepsilon$ et que tous les facteurs du membre de droite sont positifs : 
$$\delta = 
\gamma \rho_2 N^2 (M\varepsilon + \eta)^2 
+ \left(- \lambda \rho_1 + 2\rho_2 M + |||R^{-1}||| \, M^2 + \left(\frac{1}{\gamma} + k \right)\rho_2 \right) \varepsilon^2
$$

Quand $\delta_f = 0$ et $\delta_h = 0$, on a $\eta = 0$, et donc $\gamma \delta$ est un trinôme de la forme $A\gamma^2 + B\gamma + C$ où $A> 0$, $C> 0$ sont indépendants de $k$, et $B$ est fonction croissante de $k$. Pour avoir $\delta = 0$, on cherche une racine positive stricte $\gamma_1$ de ce trinôme. Pour cela il faut une valeur $k>0$ telle que le discriminant $\Delta = B^2 - 4AC \geq 0$ et $\gamma_1 = \frac{-B + \sqrt(\Delta)}{2A}  = \frac{B}{2A} \left(-1 + \sqrt(1- \frac{4AC}{B^2}) \right)> 0$ or $\gamma_1$ est donc négative dès que $B > 0$. Or la valeur de $B$ est aussi fonction de $\lambda, \rho_1,\rho_2,M,|||R^{-1}|||$ et l'on ne contrôle pas sa valeur. 

Nous proposons des valeurs de $\gamma,k$ vérifiant seulement $\delta > 0$ (et pas forcément $\delta = 0$ quand $\delta_h = 0$ et $\delta_f = 0$): $\gamma = 1 > 0, k = \lambda\frac{\rho_1}{\rho_2} > \lambda\frac{\rho_1}{\rho_2} - 1$ et $k > 0$ de sorte à ce que le deuxieme terme de droite soit $>0$



**preuve de (5):**

Montrons que $\|q(0)\| \leq \varepsilon $ implique $\forall t, \, \|q(t)\| \leq \varepsilon $

on pose pour $t \geq 0$ : $$ r(t) \hat{=} \dot{g(t)} + kg(t) \leq \delta \, (4)$$

on applique la methode de la variation de la constante à l'equation differentielle obtenue :
- solution systeme homogene : $g_h(t) = e^{-kt}$
- changement de fonction inconnue : $g = g_h \, u$
- $u$ verifie alors $r = g_h \, \dot{u}$ ie $\dot{u}(t) = r(t)e^{kt}$ donc 
$$u(t) = \int_0^t   r(s)e^{ks}ds + u(0)$$
- d'où $$<q(t),P^{-1}q(t)> = g(t) = e^{-kt}\left(\int_0^t  r(s)e^{ks}ds + u(0)\right)$$
- on encadre grâce à $(0.5)$ et par positivite des fonctions  
$$\|q(t)\|^2\rho_1 = <q(t),\rho_1I_nq(t)>\leq <q(t),P^{-1}q(t)> \leq <q(t),\rho_2I_nq(t)> = \rho_2\|q(t)\|^2 \\
\|q(t)\|^2\rho_1 \leq e^{-kt}\left(\int_0^t   r(s)e^{ks}ds +  \frac{g(0)}{g_h(0)} \right) \\
\|q(t)\|^2\leq \frac{e^{-kt}}{\rho_1 }\left(\delta \int_0^t   e^{ks}ds + \rho_2\|q(0)\|^2 \right) \\
\|q(t)\|^2\leq \frac{e^{-kt}}{\rho_1 }\left(\delta \left[\frac{e^{kt} - 1}{k}\right] + \rho_2\|q(0)\|^2 \right) \\
\|q(t)\|^2\leq \frac{\delta}{k \rho_1 }\left(1- e^{-kt}\right) + \frac{e^{-kt}\rho_2}{\rho_1 }\|q(0)\|^2 \\ 
\|q(t)\|^2\leq \frac{\delta}{k \rho_1 } + \frac{e^{-kt}\rho_2}{\rho_1 }\|q(0)\|^2 \\
\|q(t)\| \leq \sqrt{\frac{\delta}{k \rho_1 } + \frac{e^{-kt}\rho_2}{\rho_1 }\|q(0)\|^2} \\
\|q(t)\| \leq \sqrt{\frac{\delta}{k \rho_1 }} + e^{\frac{-kt}{2}}\sqrt{\frac{\rho_2}{\rho_1 }}\|q(0)\| \\
\|\hat{x}(t)-x(t) \| \leq \rho' e^{-k' t} \|\hat{x}(0)-x(0) \|  + \delta'
$$

où on pose 
- $\rho' = \sqrt{\frac{\rho_2}{\rho_1 }}$
- $k' = \frac{k}{2}$
- $\delta' = \sqrt{\frac{\delta}{k \rho_1 }}$

le $\delta'$ ainsi définit n'est pas nul quand $\eta=0$ est un majorant de $\delta_f,\delta_h$, ce qui ne répond pas à l'énoncé.

**Remarque** Notons ici que l'analyse theorique du filtre de Kalman admet souvent un cercle vicieux: la dynamique de $P$, et donc les parametres $\rho_1$ et $\rho_2$, dependent de $\hat{x}$, et on a besoin de $\rho_1$ et $\rho_2$ pour montrer la convergence de $\hat{x}$ vers $x$. Ce probleme peut être resolu dans certains cas pour des structures particulieres ou sous des hypotheses tres fortes d'observabilite uniforme. En fait, le filtre de Kalman a initialement ete developpe pour les systemes lineaires, où la dynamique de $P$ est independante de $\hat{x}$ et où les parametres $\rho_1$ et $\rho_2$ sont donc des proprietes *intrinseques* du systeme (lies à son observabilite) et independants de $\hat{x}$. La convergence de l'estimation decoule alors sans cercle vicieux, et est même *globale*, \eta'est-à-dire sans restriction sur l'erreur initiale $\|\hat{x}(0)-x(0) \|$ (ni sur les incertitudes). Vous pouvez le verifier en prenant $\kappa_f=0$ et $\kappa_h=0$. Mais vu sa grande practicite, ce filtre a ensuite ete *etendu* aux systemes nonlineaires comme decrit ci-dessus (d'où son nom, *extended Kalman filter*, "EKF"), et est l'un des filtres les plus implementes dans l'industrie (dans sa version discrete). Il est utile de connaître cet algorithme tout en ayant conscience de ses limites... Des variantes, telles que le *unscented Kalman filter* dans sa version probabiliste, permettent parfois d'obtenir de meilleurs resultats. Pour des systemes tres non lineaires, il vaut mieux utiliser des observateurs adaptes plus complexes.

Application au suivi du bateau
================

On commence par simuler precisement une trajectoire continue quelconque de bateau, que l'on considerera exacte. Bien sûr, en pratique cette solution exacte est inconnue, seule la mesure l'est, elle sera utilisee ici seulement pour comparer la solution estimee à la vraie, et evaluer ses performances.

In [12]:
def f_traj(t,x):
    v = 2.0 
    w = np.sin(t)
    return np.array([v*np.cos(x[2]),v*np.sin(x[2]),w])

In [13]:
# simulation d'une trajectoire continue à haute precision, que l'on considere exacte. 
# M'option ``dense_output`` de ``solve_ivp`` permet d'obtenir en sortie une fonction ``sol_traj.sol`` qui donne la solution 
# à n'importe quel temps (voir doc) Ceci est tres utile lorsque vous ne savez pas d'avance à quel instant vous en aurez besoin
# (comme ici la mesure, voir fonction ``y_traj`` ci-dessous)
x0 = np.array([0.0,0.0,0.0])
t0, tf = 0.0, 20.0
sol_traj = solve_ivp(fun = f_traj, t_span = [t0, tf], y0 = x0, dense_output=True, t_eval = np.arange(t0,tf,0.1), rtol=1e-4)

plt.figure()
plt.plot(sol_traj.y[0],sol_traj.y[1])
plt.grid(True)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.title('Trajectoire exacte')

def y_traj(t):
    # on selectionne les deux premieres composantes de la solution "exacte" au temps t : mesure de position exacte
    return sol_traj.sol(t)[:2] 

print(y_traj(10.0),type(y_traj(10.0)))

<IPython.core.display.Javascript object>

[ 7.37195036 13.20715265] <class 'numpy.ndarray'>


Puis on definit les parametres et fonctions necessaires à la simulation du filtre.

**Question 5** Completer ci-dessous les fonctions ``jac_f_ext`` et ``jac_h_ext`` donnant les matrices jacobiennes des fonctions de dynamique et mesure theoriques $f_{ext}:\mathbb{R}^5\to \mathbb{R}^5$ and $h_{ext}:\mathbb{R}^5\to \mathbb{R}^2$ donnees par
$$
f_{ext}(x) = (x_4 \cos(x_3),x_4 \sin(x_3),x_5 ,0,0) \quad , \quad h_{ext} = (x_1,x_2)
$$
Vu que les incertitudes $\delta_v,\delta_w,\delta_y$ sont inconnues, ce sont ces fonctions qui seront utilisees dans le filtre.

In [14]:
def mat_mul(mat_list):
    """calcule le produit matriciel x[0] @ x[1] @ ... @ x[M]"""
    if mat_list == []:
        return 1 # np.dot effectue le produit par un scalaire
    else:
        a = mat_list.pop()
        return np.dot(mat_mul(mat_list), a)

In [15]:
# scenario de mesure
std_y = 1.0  # ecart-type du bruit de mesure

# parametres du filtre de Kalman
# n = 5, r = 2

R = np.eye(2)
Rinv = np.linalg.inv(R)
Q = np.eye(5)
lbda = 1
xhat0 = np.array([0.,0.,0.,0.,0.]) # to be modified
P0 = np.eye(5)
xhatP0 = np.hstack([xhat0, P0[0, 0], P0[0, 1], P0[0, 2], P0[0, 3], P0[0, 4], P0[1, 1],
                   P0[1, 2], P0[1, 3], P0[1, 4], P0[2, 2], P0[2, 3], P0[2, 4], P0[3, 3], P0[3, 4], P0[4, 4]])
# stack inline xhat0 and P0
# print("p0", xhatP0)


def jac_f_ext(t, x):
    # jacobian matrix of the dynamics f_ext
    # t does not seem to be used
    return np.array([
        [0., 0., -x[3]*np.sin(x[2]), np.cos(x[2]), 0.],
        [0., 0., x[3]*np.cos(x[2]), np.sin(x[2]), 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]
    ])


# print(jac_f_ext(0, xhat0))


def jac_h_ext(t, x):
    # jacobian matrix of the output map h_ext = (x1,x2)
    return np.array([
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.]
    ])


# print(jac_h_ext(0, xhat0))


def f_EKF(t, xhatP):
    """inputs : t, xhat(t) and P(t), outputs : xhat_dot(t) and P_dot(t)"""
    xhat = xhatP[:5]  # h_ext(x) = xhat[:2]
    Ps = xhatP[5:]
    P = np.array([[Ps[0], Ps[1], Ps[2], Ps[3], Ps[4]],
                  [Ps[1], Ps[5], Ps[6], Ps[7], Ps[8]],
                  [Ps[2], Ps[6], Ps[9], Ps[10], Ps[11]],
                  [Ps[3], Ps[7], Ps[10], Ps[12], Ps[13]],
                  [Ps[4], Ps[8], Ps[11], Ps[13], Ps[14]]])
    jac_f = jac_f_ext(t, xhat)
    jac_h = jac_h_ext(t, xhat)
    
    # enlever le facteur zero pour ajouter le bruit
    y_mes = y_traj(t) + np.random.normal(0.0, std_y)
    
    prod1 = mat_mul([P, np.transpose(jac_h), Rinv, y_mes - xhat[:2]])
    # print("prod1",prod1,type(prod1))
    
    xhat_dot =  f_ext(t,xhat) + prod1
    # print("xhat_dot",xhat_dot)
    
    prod2 = mat_mul([P, np.transpose(jac_h), Rinv, jac_h, P])
    # print("prod2",prod2,type(prod2))
    P_dot = lbda*P + np.dot(jac_f, P) + np.dot(P, np.transpose(jac_f)) + Q - prod2
    
    # return xhat_dot, P_dot[0,0]
    return np.hstack([xhat_dot, P_dot[0, 0], P_dot[0, 1], P_dot[0, 2], P_dot[0, 3], P_dot[0, 4], P_dot[1, 1], P_dot[1, 2], P_dot[1, 3], P_dot[1, 4], P_dot[2, 2], P_dot[2, 3], P_dot[2, 4], P_dot[3, 3], P_dot[3, 4], P_dot[4, 4]])


# print(f_EKF(10, xhatP0))


**Question 6** Completer la fonction ``f_EKF`` pour simuler la dynamique du filtre de Kalman etendu.

**Question 7** Implementer le filtre en appliquant votre fonction ``solve_euler_explicit`` à ``f_EKF``. 

Comparer l'estimation obtenue avec la trajectoire exacte pour differentes valeurs de parametres $(R,Q,\lambda)$ et conditions initiales, d'abord en absence de bruit de mesure, puis en introduisant le bruit. 

Que se passe-t-il lorsque $R$ est pris petit/grand ? 

Justifier qu'on relie ce parametre à la confiance que l'on a en la mesure. 

Lorsqu'on rajoute le bruit de mesure, expliquer pourquoi l'on dit souvent que le filtre fait un compromis entre bruit de modele et bruit de mesure.

In [16]:
# sur la plage de temps [t0,tf], avec un resolution temporelle dt = 1, les coefficients des matrices & vecteurs divergent 
# et deviennent des +-inf puis des nan ce qui leve une erreur
# pour empêcher cela, il faut utiliser un dt <= 0.001


t_list, xhatP_list = solve_euler_explicit(f_EKF, xhatP0, .001, t0, tf)
xhat_list = xhatP_list[:,:5] # to be sliced, we need the first 5 coords of each element
plt.figure()
plt.plot(xhat_list[:,0],xhat_list[:,1])
plt.grid(True)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')


<IPython.core.display.Javascript object>

Text(0, 0.5, '$x_2$')

## Variation de R, avec  bruit

In [17]:
# scenario de mesure
std_y = 1.0  # ecart-type du bruit de mesure

# parametres du filtre de Kalman
# n = 5, r = 2

R =  1000 * np.eye(2)
Rinv = np.linalg.inv(R)
Q = np.eye(5)
lbda = 1
xhat0 = np.array([0.,0.,0.,0.,0.]) # to be modified
P0 = np.eye(5)
xhatP0 = np.hstack([xhat0, P0[0, 0], P0[0, 1], P0[0, 2], P0[0, 3], P0[0, 4], P0[1, 1],
                   P0[1, 2], P0[1, 3], P0[1, 4], P0[2, 2], P0[2, 3], P0[2, 4], P0[3, 3], P0[3, 4], P0[4, 4]])
# stack inline xhat0 and P0
# print("p0", xhatP0)


def jac_f_ext(t, x):
    # jacobian matrix of the dynamics f_ext
    # t does not seem to be used
    return np.array([
        [0., 0., -x[3]*np.sin(x[2]), np.cos(x[2]), 0.],
        [0., 0., x[3]*np.cos(x[2]), np.sin(x[2]), 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]
    ])


# print(jac_f_ext(0, xhat0))


def jac_h_ext(t, x):
    # jacobian matrix of the output map h_ext = (x1,x2)
    return np.array([
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.]
    ])


# print(jac_h_ext(0, xhat0))


def f_EKF(t, xhatP):
    """inputs : t, xhat(t) and P(t), outputs : xhat_dot(t) and P_dot(t)"""
    xhat = xhatP[:5]  # h_ext(x) = xhat[:2]
    Ps = xhatP[5:]
    P = np.array([[Ps[0], Ps[1], Ps[2], Ps[3], Ps[4]],
                  [Ps[1], Ps[5], Ps[6], Ps[7], Ps[8]],
                  [Ps[2], Ps[6], Ps[9], Ps[10], Ps[11]],
                  [Ps[3], Ps[7], Ps[10], Ps[12], Ps[13]],
                  [Ps[4], Ps[8], Ps[11], Ps[13], Ps[14]]])
    jac_f = jac_f_ext(t, xhat)
    jac_h = jac_h_ext(t, xhat)
    
    # enlever le facteur zero pour ajouter le bruit
    y_mes = y_traj(t) + np.random.normal(0.0, std_y)
    
    prod1 = mat_mul([P, np.transpose(jac_h), Rinv, y_mes - xhat[:2]])
    # print("prod1",prod1,type(prod1))
    
    xhat_dot =  f_ext(t,xhat) + prod1
    # print("xhat_dot",xhat_dot)
    
    prod2 = mat_mul([P, np.transpose(jac_h), Rinv, jac_h, P])
    # print("prod2",prod2,type(prod2))
    P_dot = lbda*P + np.dot(jac_f, P) + np.dot(P, np.transpose(jac_f)) + Q - prod2
    
    # return xhat_dot, P_dot[0,0]
    return np.hstack([xhat_dot, P_dot[0, 0], P_dot[0, 1], P_dot[0, 2], P_dot[0, 3], P_dot[0, 4], P_dot[1, 1], P_dot[1, 2], P_dot[1, 3], P_dot[1, 4], P_dot[2, 2], P_dot[2, 3], P_dot[2, 4], P_dot[3, 3], P_dot[3, 4], P_dot[4, 4]])


# print(f_EKF(10, xhatP0))

t_list, xhatP_list = solve_euler_explicit(f_EKF, xhatP0, .001, t0, tf)
xhat_list = xhatP_list[:,:5] # to be sliced, we need the first 5 coords of each element
plt.figure()
plt.plot(xhat_list[:,0],xhat_list[:,1])
plt.grid(True)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')


<IPython.core.display.Javascript object>

Text(0, 0.5, '$x_2$')

**Analyse :**
Quand $R$ a une "grande" norme subordonnée (par exemple une homothétie de "grand" facteur), le filtre met plus de temps à approximer correctement la trajectoire réelle (donc grand bruit de modèle aux temps courts) mais le bruit de mesure est peu visible .

In [18]:
# scenario de mesure
std_y = 1.0  # ecart-type du bruit de mesure

# parametres du filtre de Kalman
# n = 5, r = 2

R =  0.001 * np.eye(2)
Rinv = np.linalg.inv(R)
Q = np.eye(5)
lbda = 1
xhat0 = np.array([0.,0.,0.,0.,0.]) # to be modified
P0 = np.eye(5)
xhatP0 = np.hstack([xhat0, P0[0, 0], P0[0, 1], P0[0, 2], P0[0, 3], P0[0, 4], P0[1, 1],
                   P0[1, 2], P0[1, 3], P0[1, 4], P0[2, 2], P0[2, 3], P0[2, 4], P0[3, 3], P0[3, 4], P0[4, 4]])
# stack inline xhat0 and P0
# print("p0", xhatP0)


def jac_f_ext(t, x):
    # jacobian matrix of the dynamics f_ext
    # t does not seem to be used
    return np.array([
        [0., 0., -x[3]*np.sin(x[2]), np.cos(x[2]), 0.],
        [0., 0., x[3]*np.cos(x[2]), np.sin(x[2]), 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]
    ])


# print(jac_f_ext(0, xhat0))


def jac_h_ext(t, x):
    # jacobian matrix of the output map h_ext = (x1,x2)
    return np.array([
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.]
    ])


# print(jac_h_ext(0, xhat0))


def f_EKF(t, xhatP):
    """inputs : t, xhat(t) and P(t), outputs : xhat_dot(t) and P_dot(t)"""
    xhat = xhatP[:5]  # h_ext(x) = xhat[:2]
    Ps = xhatP[5:]
    P = np.array([[Ps[0], Ps[1], Ps[2], Ps[3], Ps[4]],
                  [Ps[1], Ps[5], Ps[6], Ps[7], Ps[8]],
                  [Ps[2], Ps[6], Ps[9], Ps[10], Ps[11]],
                  [Ps[3], Ps[7], Ps[10], Ps[12], Ps[13]],
                  [Ps[4], Ps[8], Ps[11], Ps[13], Ps[14]]])
    jac_f = jac_f_ext(t, xhat)
    jac_h = jac_h_ext(t, xhat)
    
    # enlever le facteur zero pour ajouter le bruit
    y_mes = y_traj(t) + 1* np.random.normal(0.0, std_y)
    
    prod1 = mat_mul([P, np.transpose(jac_h), Rinv, y_mes - xhat[:2]])
    # print("prod1",prod1,type(prod1))
    
    xhat_dot =  f_ext(t,xhat) + prod1
    # print("xhat_dot",xhat_dot)
    
    prod2 = mat_mul([P, np.transpose(jac_h), Rinv, jac_h, P])
    # print("prod2",prod2,type(prod2))
    P_dot = lbda*P + np.dot(jac_f, P) + np.dot(P, np.transpose(jac_f)) + Q - prod2
    
    # return xhat_dot, P_dot[0,0]
    return np.hstack([xhat_dot, P_dot[0, 0], P_dot[0, 1], P_dot[0, 2], P_dot[0, 3], P_dot[0, 4], P_dot[1, 1], P_dot[1, 2], P_dot[1, 3], P_dot[1, 4], P_dot[2, 2], P_dot[2, 3], P_dot[2, 4], P_dot[3, 3], P_dot[3, 4], P_dot[4, 4]])


# print(f_EKF(10, xhatP0))

t_list, xhatP_list = solve_euler_explicit(f_EKF, xhatP0, .001, t0, tf)
xhat_list = xhatP_list[:,:5] # to be sliced, we need the first 5 coords of each element
plt.figure()
plt.plot(xhat_list[:,0],xhat_list[:,1])
plt.grid(True)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')


<IPython.core.display.Javascript object>

Text(0, 0.5, '$x_2$')

**Analyse :**
Quand $R$ a une "petite" norme subordonnée (par exemple une homothétie de "petit" facteur), le filtre approxime plus rapidement la trajectoire réelle aux temps  courts (donc a un faible bruit de modèle), mais le bruit de mesure est très visible sur l'estimation du filtre .

## Confiance en la mesure 
Au vu des résultats, si l'on estime que le bruit de mesure n'est pas gênant, et donc qu'on a confiance en la mesure, on peut se permettre de prendre un $R$ de petite norme pour diminuer le bruit de modèle.

## Variation de Q, avec bruit

In [19]:
# scenario de mesure
std_y = 1.0  # ecart-type du bruit de mesure

# parametres du filtre de Kalman
# n = 5, r = 2

R =  np.eye(2)
Rinv = np.linalg.inv(R)
Q = 1000 * np.eye(5)
lbda = 1
xhat0 = np.array([0.,0.,0.,0.,0.]) # to be modified
P0 = np.eye(5)
xhatP0 = np.hstack([xhat0, P0[0, 0], P0[0, 1], P0[0, 2], P0[0, 3], P0[0, 4], P0[1, 1],
                   P0[1, 2], P0[1, 3], P0[1, 4], P0[2, 2], P0[2, 3], P0[2, 4], P0[3, 3], P0[3, 4], P0[4, 4]])
# stack inline xhat0 and P0
# print("p0", xhatP0)


def jac_f_ext(t, x):
    # jacobian matrix of the dynamics f_ext
    # t does not seem to be used
    return np.array([
        [0., 0., -x[3]*np.sin(x[2]), np.cos(x[2]), 0.],
        [0., 0., x[3]*np.cos(x[2]), np.sin(x[2]), 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]
    ])


# print(jac_f_ext(0, xhat0))


def jac_h_ext(t, x):
    # jacobian matrix of the output map h_ext = (x1,x2)
    return np.array([
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.]
    ])


# print(jac_h_ext(0, xhat0))


def f_EKF(t, xhatP):
    """inputs : t, xhat(t) and P(t), outputs : xhat_dot(t) and P_dot(t)"""
    xhat = xhatP[:5]  # h_ext(x) = xhat[:2]
    Ps = xhatP[5:]
    P = np.array([[Ps[0], Ps[1], Ps[2], Ps[3], Ps[4]],
                  [Ps[1], Ps[5], Ps[6], Ps[7], Ps[8]],
                  [Ps[2], Ps[6], Ps[9], Ps[10], Ps[11]],
                  [Ps[3], Ps[7], Ps[10], Ps[12], Ps[13]],
                  [Ps[4], Ps[8], Ps[11], Ps[13], Ps[14]]])
    jac_f = jac_f_ext(t, xhat)
    jac_h = jac_h_ext(t, xhat)
    
    # enlever le facteur zero pour ajouter le bruit
    y_mes = y_traj(t) + np.random.normal(0.0, std_y)
    
    prod1 = mat_mul([P, np.transpose(jac_h), Rinv, y_mes - xhat[:2]])
    # print("prod1",prod1,type(prod1))
    
    xhat_dot =  f_ext(t,xhat) + prod1
    # print("xhat_dot",xhat_dot)
    
    prod2 = mat_mul([P, np.transpose(jac_h), Rinv, jac_h, P])
    # print("prod2",prod2,type(prod2))
    P_dot = lbda*P + np.dot(jac_f, P) + np.dot(P, np.transpose(jac_f)) + Q - prod2
    
    # return xhat_dot, P_dot[0,0]
    return np.hstack([xhat_dot, P_dot[0, 0], P_dot[0, 1], P_dot[0, 2], P_dot[0, 3], P_dot[0, 4], P_dot[1, 1], P_dot[1, 2], P_dot[1, 3], P_dot[1, 4], P_dot[2, 2], P_dot[2, 3], P_dot[2, 4], P_dot[3, 3], P_dot[3, 4], P_dot[4, 4]])


# print(f_EKF(10, xhatP0))

t_list, xhatP_list = solve_euler_explicit(f_EKF, xhatP0, .001, t0, tf)
xhat_list = xhatP_list[:,:5] # to be sliced, we need the first 5 coords of each element
plt.figure()
plt.plot(xhat_list[:,0],xhat_list[:,1])
plt.grid(True)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')


<IPython.core.display.Javascript object>

Text(0, 0.5, '$x_2$')

In [20]:
# scenario de mesure
std_y = 1.0  # ecart-type du bruit de mesure

# parametres du filtre de Kalman
# n = 5, r = 2

R =  np.eye(2)
Rinv = np.linalg.inv(R)
Q = 0.001 * np.eye(5)
lbda = 1
xhat0 = np.array([0.,0.,0.,0.,0.]) # to be modified
P0 = np.eye(5)
xhatP0 = np.hstack([xhat0, P0[0, 0], P0[0, 1], P0[0, 2], P0[0, 3], P0[0, 4], P0[1, 1],
                   P0[1, 2], P0[1, 3], P0[1, 4], P0[2, 2], P0[2, 3], P0[2, 4], P0[3, 3], P0[3, 4], P0[4, 4]])
# stack inline xhat0 and P0
# print("p0", xhatP0)


def jac_f_ext(t, x):
    # jacobian matrix of the dynamics f_ext
    # t does not seem to be used
    return np.array([
        [0., 0., -x[3]*np.sin(x[2]), np.cos(x[2]), 0.],
        [0., 0., x[3]*np.cos(x[2]), np.sin(x[2]), 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]
    ])


# print(jac_f_ext(0, xhat0))


def jac_h_ext(t, x):
    # jacobian matrix of the output map h_ext = (x1,x2)
    return np.array([
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.]
    ])


# print(jac_h_ext(0, xhat0))


def f_EKF(t, xhatP):
    """inputs : t, xhat(t) and P(t), outputs : xhat_dot(t) and P_dot(t)"""
    xhat = xhatP[:5]  # h_ext(x) = xhat[:2]
    Ps = xhatP[5:]
    P = np.array([[Ps[0], Ps[1], Ps[2], Ps[3], Ps[4]],
                  [Ps[1], Ps[5], Ps[6], Ps[7], Ps[8]],
                  [Ps[2], Ps[6], Ps[9], Ps[10], Ps[11]],
                  [Ps[3], Ps[7], Ps[10], Ps[12], Ps[13]],
                  [Ps[4], Ps[8], Ps[11], Ps[13], Ps[14]]])
    jac_f = jac_f_ext(t, xhat)
    jac_h = jac_h_ext(t, xhat)
    
    # enlever le facteur zero pour ajouter le bruit
    y_mes = y_traj(t) + np.random.normal(0.0, std_y)
    
    prod1 = mat_mul([P, np.transpose(jac_h), Rinv, y_mes - xhat[:2]])
    # print("prod1",prod1,type(prod1))
    
    xhat_dot =  f_ext(t,xhat) + prod1
    # print("xhat_dot",xhat_dot)
    
    prod2 = mat_mul([P, np.transpose(jac_h), Rinv, jac_h, P])
    # print("prod2",prod2,type(prod2))
    P_dot = lbda*P + np.dot(jac_f, P) + np.dot(P, np.transpose(jac_f)) + Q - prod2
    
    # return xhat_dot, P_dot[0,0]
    return np.hstack([xhat_dot, P_dot[0, 0], P_dot[0, 1], P_dot[0, 2], P_dot[0, 3], P_dot[0, 4], P_dot[1, 1], P_dot[1, 2], P_dot[1, 3], P_dot[1, 4], P_dot[2, 2], P_dot[2, 3], P_dot[2, 4], P_dot[3, 3], P_dot[3, 4], P_dot[4, 4]])


# print(f_EKF(10, xhatP0))

t_list, xhatP_list = solve_euler_explicit(f_EKF, xhatP0, .001, t0, tf)
xhat_list = xhatP_list[:,:5] # to be sliced, we need the first 5 coords of each element
plt.figure()
plt.plot(xhat_list[:,0],xhat_list[:,1])
plt.grid(True)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')


<IPython.core.display.Javascript object>

Text(0, 0.5, '$x_2$')

**Analyse :**
Contrairement à $R$, $Q$ a uniquement un impact sur la visiblité du bruit de mesure, celle-ci augmente avec la norme subordonnée de $Q$. 

## Compromis entre bruit de modèle et bruit de mesure
Pour diminuer la visiblité du bruit de mesure sur l'estimation, on peut augmenter la norme de $R$, mais cela augmente le bruit de modèle, ie l'estimation approxime moins bien la trajectoire réelle aux temps courts. Pour diminuer le bruit de modèle, on peut diminuer la norme de $R$, ce qui rend le bruit de mesure plus visible. Pour contrôler ce dernier on peut diminuer la norme de $Q$, mais cela a ses limites, car la diminution de $R$ entraîne forcément une divergence du bruit de mesure visible sur l'estimation.

## Variation de $\lambda$  avec bruit

In [21]:
# scenario de mesure
std_y = 1.0  # ecart-type du bruit de mesure

# parametres du filtre de Kalman
# n = 5, r = 2

R =  np.eye(2)
Rinv = np.linalg.inv(R)
Q = np.eye(5)
lbda = 10
xhat0 = np.array([0.,0.,0.,0.,0.]) # to be modified
P0 = np.eye(5)
xhatP0 = np.hstack([xhat0, P0[0, 0], P0[0, 1], P0[0, 2], P0[0, 3], P0[0, 4], P0[1, 1],
                   P0[1, 2], P0[1, 3], P0[1, 4], P0[2, 2], P0[2, 3], P0[2, 4], P0[3, 3], P0[3, 4], P0[4, 4]])
# stack inline xhat0 and P0
# print("p0", xhatP0)


def jac_f_ext(t, x):
    # jacobian matrix of the dynamics f_ext
    # t does not seem to be used
    return np.array([
        [0., 0., -x[3]*np.sin(x[2]), np.cos(x[2]), 0.],
        [0., 0., x[3]*np.cos(x[2]), np.sin(x[2]), 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]
    ])


# print(jac_f_ext(0, xhat0))


def jac_h_ext(t, x):
    # jacobian matrix of the output map h_ext = (x1,x2)
    return np.array([
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.]
    ])


# print(jac_h_ext(0, xhat0))


def f_EKF(t, xhatP):
    """inputs : t, xhat(t) and P(t), outputs : xhat_dot(t) and P_dot(t)"""
    xhat = xhatP[:5]  # h_ext(x) = xhat[:2]
    Ps = xhatP[5:]
    P = np.array([[Ps[0], Ps[1], Ps[2], Ps[3], Ps[4]],
                  [Ps[1], Ps[5], Ps[6], Ps[7], Ps[8]],
                  [Ps[2], Ps[6], Ps[9], Ps[10], Ps[11]],
                  [Ps[3], Ps[7], Ps[10], Ps[12], Ps[13]],
                  [Ps[4], Ps[8], Ps[11], Ps[13], Ps[14]]])
    jac_f = jac_f_ext(t, xhat)
    jac_h = jac_h_ext(t, xhat)
    
    # enlever le facteur zero pour ajouter le bruit
    y_mes = y_traj(t) + np.random.normal(0.0, std_y)
    
    prod1 = mat_mul([P, np.transpose(jac_h), Rinv, y_mes - xhat[:2]])
    # print("prod1",prod1,type(prod1))
    
    xhat_dot =  f_ext(t,xhat) + prod1
    # print("xhat_dot",xhat_dot)
    
    prod2 = mat_mul([P, np.transpose(jac_h), Rinv, jac_h, P])
    # print("prod2",prod2,type(prod2))
    P_dot = lbda*P + np.dot(jac_f, P) + np.dot(P, np.transpose(jac_f)) + Q - prod2
    
    # return xhat_dot, P_dot[0,0]
    return np.hstack([xhat_dot, P_dot[0, 0], P_dot[0, 1], P_dot[0, 2], P_dot[0, 3], P_dot[0, 4], P_dot[1, 1], P_dot[1, 2], P_dot[1, 3], P_dot[1, 4], P_dot[2, 2], P_dot[2, 3], P_dot[2, 4], P_dot[3, 3], P_dot[3, 4], P_dot[4, 4]])


# print(f_EKF(10, xhatP0))

t_list, xhatP_list = solve_euler_explicit(f_EKF, xhatP0, .001, t0, tf)
xhat_list = xhatP_list[:,:5] # to be sliced, we need the first 5 coords of each element
plt.figure()
plt.plot(xhat_list[:,0],xhat_list[:,1])
plt.grid(True)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')


C:\Users\Jules\AppData\Local\Temp\ipykernel_13256\2217568876.py:23: RuntimeWarning: invalid value encountered in sin
  [0., 0., -x[3]*np.sin(x[2]), np.cos(x[2]), 0.],
C:\Users\Jules\AppData\Local\Temp\ipykernel_13256\2217568876.py:23: RuntimeWarning: invalid value encountered in cos
  [0., 0., -x[3]*np.sin(x[2]), np.cos(x[2]), 0.],
C:\Users\Jules\AppData\Local\Temp\ipykernel_13256\2217568876.py:24: RuntimeWarning: invalid value encountered in cos
  [0., 0., x[3]*np.cos(x[2]), np.sin(x[2]), 0.],
C:\Users\Jules\AppData\Local\Temp\ipykernel_13256\2217568876.py:24: RuntimeWarning: invalid value encountered in sin
  [0., 0., x[3]*np.cos(x[2]), np.sin(x[2]), 0.],
C:\Users\Jules\AppData\Local\Temp\ipykernel_13256\2597625584.py:2: RuntimeWarning: invalid value encountered in cos
  return np.array([x[3]*np.cos(x[2]),x[3]*np.sin(x[2]),x[4],0,0])
C:\Users\Jules\AppData\Local\Temp\ipykernel_13256\2597625584.py:2: RuntimeWarning: invalid value encountered in sin
  return np.array([x[3]*np.cos(x[2])

<IPython.core.display.Javascript object>

Text(0, 0.5, '$x_2$')

In [22]:
# scenario de mesure
std_y = 1.0  # ecart-type du bruit de mesure

# parametres du filtre de Kalman
# n = 5, r = 2

R =  np.eye(2)
Rinv = np.linalg.inv(R)
Q = np.eye(5)
lbda = 0.0001
xhat0 = np.array([0.,0.,0.,0.,0.]) # to be modified
P0 = np.eye(5)
xhatP0 = np.hstack([xhat0, P0[0, 0], P0[0, 1], P0[0, 2], P0[0, 3], P0[0, 4], P0[1, 1],
                   P0[1, 2], P0[1, 3], P0[1, 4], P0[2, 2], P0[2, 3], P0[2, 4], P0[3, 3], P0[3, 4], P0[4, 4]])
# stack inline xhat0 and P0
# print("p0", xhatP0)


def jac_f_ext(t, x):
    # jacobian matrix of the dynamics f_ext
    # t does not seem to be used
    return np.array([
        [0., 0., -x[3]*np.sin(x[2]), np.cos(x[2]), 0.],
        [0., 0., x[3]*np.cos(x[2]), np.sin(x[2]), 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]
    ])


# print(jac_f_ext(0, xhat0))


def jac_h_ext(t, x):
    # jacobian matrix of the output map h_ext = (x1,x2)
    return np.array([
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.]
    ])


# print(jac_h_ext(0, xhat0))


def f_EKF(t, xhatP):
    """inputs : t, xhat(t) and P(t), outputs : xhat_dot(t) and P_dot(t)"""
    xhat = xhatP[:5]  # h_ext(x) = xhat[:2]
    Ps = xhatP[5:]
    P = np.array([[Ps[0], Ps[1], Ps[2], Ps[3], Ps[4]],
                  [Ps[1], Ps[5], Ps[6], Ps[7], Ps[8]],
                  [Ps[2], Ps[6], Ps[9], Ps[10], Ps[11]],
                  [Ps[3], Ps[7], Ps[10], Ps[12], Ps[13]],
                  [Ps[4], Ps[8], Ps[11], Ps[13], Ps[14]]])
    jac_f = jac_f_ext(t, xhat)
    jac_h = jac_h_ext(t, xhat)
    
    # enlever le facteur zero pour ajouter le bruit
    y_mes = y_traj(t) + np.random.normal(0.0, std_y)
    
    prod1 = mat_mul([P, np.transpose(jac_h), Rinv, y_mes - xhat[:2]])
    # print("prod1",prod1,type(prod1))
    
    xhat_dot =  f_ext(t,xhat) + prod1
    # print("xhat_dot",xhat_dot)
    
    prod2 = mat_mul([P, np.transpose(jac_h), Rinv, jac_h, P])
    # print("prod2",prod2,type(prod2))
    P_dot = lbda*P + np.dot(jac_f, P) + np.dot(P, np.transpose(jac_f)) + Q - prod2
    
    # return xhat_dot, P_dot[0,0]
    return np.hstack([xhat_dot, P_dot[0, 0], P_dot[0, 1], P_dot[0, 2], P_dot[0, 3], P_dot[0, 4], P_dot[1, 1], P_dot[1, 2], P_dot[1, 3], P_dot[1, 4], P_dot[2, 2], P_dot[2, 3], P_dot[2, 4], P_dot[3, 3], P_dot[3, 4], P_dot[4, 4]])


# print(f_EKF(10, xhatP0))

t_list, xhatP_list = solve_euler_explicit(f_EKF, xhatP0, .001, t0, tf)
xhat_list = xhatP_list[:,:5] # to be sliced, we need the first 5 coords of each element
plt.figure()
plt.plot(xhat_list[:,0],xhat_list[:,1])
plt.grid(True)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')


<IPython.core.display.Javascript object>

Text(0, 0.5, '$x_2$')

In [23]:
# scenario de mesure
std_y = 1.0  # ecart-type du bruit de mesure

# parametres du filtre de Kalman
# n = 5, r = 2

R =  np.eye(2)
Rinv = np.linalg.inv(R)
Q = 0.001 * np.eye(5)
lbda = 0
xhat0 = np.array([0.,0.,0.,0.,0.]) # to be modified
P0 = np.eye(5)
xhatP0 = np.hstack([xhat0, P0[0, 0], P0[0, 1], P0[0, 2], P0[0, 3], P0[0, 4], P0[1, 1],
                   P0[1, 2], P0[1, 3], P0[1, 4], P0[2, 2], P0[2, 3], P0[2, 4], P0[3, 3], P0[3, 4], P0[4, 4]])
# stack inline xhat0 and P0
# print("p0", xhatP0)


def jac_f_ext(t, x):
    # jacobian matrix of the dynamics f_ext
    # t does not seem to be used
    return np.array([
        [0., 0., -x[3]*np.sin(x[2]), np.cos(x[2]), 0.],
        [0., 0., x[3]*np.cos(x[2]), np.sin(x[2]), 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]
    ])


# print(jac_f_ext(0, xhat0))


def jac_h_ext(t, x):
    # jacobian matrix of the output map h_ext = (x1,x2)
    return np.array([
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.]
    ])


# print(jac_h_ext(0, xhat0))


def f_EKF(t, xhatP):
    """inputs : t, xhat(t) and P(t), outputs : xhat_dot(t) and P_dot(t)"""
    xhat = xhatP[:5]  # h_ext(x) = xhat[:2]
    Ps = xhatP[5:]
    P = np.array([[Ps[0], Ps[1], Ps[2], Ps[3], Ps[4]],
                  [Ps[1], Ps[5], Ps[6], Ps[7], Ps[8]],
                  [Ps[2], Ps[6], Ps[9], Ps[10], Ps[11]],
                  [Ps[3], Ps[7], Ps[10], Ps[12], Ps[13]],
                  [Ps[4], Ps[8], Ps[11], Ps[13], Ps[14]]])
    jac_f = jac_f_ext(t, xhat)
    jac_h = jac_h_ext(t, xhat)
    
    # enlever le facteur zero pour ajouter le bruit
    y_mes = y_traj(t) + np.random.normal(0.0, std_y)
    
    prod1 = mat_mul([P, np.transpose(jac_h), Rinv, y_mes - xhat[:2]])
    # print("prod1",prod1,type(prod1))
    
    xhat_dot =  f_ext(t,xhat) + prod1
    # print("xhat_dot",xhat_dot)
    
    prod2 = mat_mul([P, np.transpose(jac_h), Rinv, jac_h, P])
    # print("prod2",prod2,type(prod2))
    P_dot = lbda*P + np.dot(jac_f, P) + np.dot(P, np.transpose(jac_f)) + Q - prod2
    
    # return xhat_dot, P_dot[0,0]
    return np.hstack([xhat_dot, P_dot[0, 0], P_dot[0, 1], P_dot[0, 2], P_dot[0, 3], P_dot[0, 4], P_dot[1, 1], P_dot[1, 2], P_dot[1, 3], P_dot[1, 4], P_dot[2, 2], P_dot[2, 3], P_dot[2, 4], P_dot[3, 3], P_dot[3, 4], P_dot[4, 4]])


# print(f_EKF(10, xhatP0))

t_list, xhatP_list = solve_euler_explicit(f_EKF, xhatP0, .001, t0, tf)
xhat_list = xhatP_list[:,:5] # to be sliced, we need the first 5 coords of each element
plt.figure()
plt.plot(xhat_list[:,0],xhat_list[:,1])
plt.grid(True)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')


<IPython.core.display.Javascript object>

Text(0, 0.5, '$x_2$')

**Analyse :**
L'augmentation de $\lambda$ entraîne l'échec du filtre, probablement à cause d'une divergence des valeurs mesurées. La diminution de $\lambda > 0$ quand le filtre aboutit, n'a pas d'impact visible sur l'estimation. Par contre, pour $\lambda = 0$, on observe qu'aux temps longs, l'estimation du filtre s'éloigne de la trajectoire réelle.

## Variation des conditions initiales

In [24]:
# scenario de mesure
std_y = 1.0  # ecart-type du bruit de mesure

# parametres du filtre de Kalman
# n = 5, r = 2

R =  np.eye(2)
Rinv = np.linalg.inv(R)
Q = np.eye(5)
lbda = 1
xhat0 = np.array([0.,0.,0.,0.,0.]) # to be modified
P0 = 0.0001*np.eye(5)
xhatP0 = np.hstack([xhat0, P0[0, 0], P0[0, 1], P0[0, 2], P0[0, 3], P0[0, 4], P0[1, 1],
                   P0[1, 2], P0[1, 3], P0[1, 4], P0[2, 2], P0[2, 3], P0[2, 4], P0[3, 3], P0[3, 4], P0[4, 4]])
# stack inline xhat0 and P0
# print("p0", xhatP0)


def jac_f_ext(t, x):
    # jacobian matrix of the dynamics f_ext
    # t does not seem to be used
    return np.array([
        [0., 0., -x[3]*np.sin(x[2]), np.cos(x[2]), 0.],
        [0., 0., x[3]*np.cos(x[2]), np.sin(x[2]), 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]
    ])


# print(jac_f_ext(0, xhat0))


def jac_h_ext(t, x):
    # jacobian matrix of the output map h_ext = (x1,x2)
    return np.array([
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.]
    ])


# print(jac_h_ext(0, xhat0))


def f_EKF(t, xhatP):
    """inputs : t, xhat(t) and P(t), outputs : xhat_dot(t) and P_dot(t)"""
    xhat = xhatP[:5]  # h_ext(x) = xhat[:2]
    Ps = xhatP[5:]
    P = np.array([[Ps[0], Ps[1], Ps[2], Ps[3], Ps[4]],
                  [Ps[1], Ps[5], Ps[6], Ps[7], Ps[8]],
                  [Ps[2], Ps[6], Ps[9], Ps[10], Ps[11]],
                  [Ps[3], Ps[7], Ps[10], Ps[12], Ps[13]],
                  [Ps[4], Ps[8], Ps[11], Ps[13], Ps[14]]])
    jac_f = jac_f_ext(t, xhat)
    jac_h = jac_h_ext(t, xhat)
    
    # enlever le facteur zero pour ajouter le bruit
    y_mes = y_traj(t) + np.random.normal(0.0, std_y)
    
    prod1 = mat_mul([P, np.transpose(jac_h), Rinv, y_mes - xhat[:2]])
    # print("prod1",prod1,type(prod1))
    
    xhat_dot =  f_ext(t,xhat) + prod1
    # print("xhat_dot",xhat_dot)
    
    prod2 = mat_mul([P, np.transpose(jac_h), Rinv, jac_h, P])
    # print("prod2",prod2,type(prod2))
    P_dot = lbda*P + np.dot(jac_f, P) + np.dot(P, np.transpose(jac_f)) + Q - prod2
    
    # return xhat_dot, P_dot[0,0]
    return np.hstack([xhat_dot, P_dot[0, 0], P_dot[0, 1], P_dot[0, 2], P_dot[0, 3], P_dot[0, 4], P_dot[1, 1], P_dot[1, 2], P_dot[1, 3], P_dot[1, 4], P_dot[2, 2], P_dot[2, 3], P_dot[2, 4], P_dot[3, 3], P_dot[3, 4], P_dot[4, 4]])


# print(f_EKF(10, xhatP0))

t_list, xhatP_list = solve_euler_explicit(f_EKF, xhatP0, .001, t0, tf)
xhat_list = xhatP_list[:,:5] # to be sliced, we need the first 5 coords of each element
plt.figure()
plt.plot(xhat_list[:,0],xhat_list[:,1])
plt.grid(True)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')


<IPython.core.display.Javascript object>

Text(0, 0.5, '$x_2$')

In [25]:
# scenario de mesure
std_y = 1.0  # ecart-type du bruit de mesure

# parametres du filtre de Kalman
# n = 5, r = 2

R =  np.eye(2)
Rinv = np.linalg.inv(R)
Q = np.eye(5)
lbda = 1
xhat0 = np.array([0.,0.,0.,0.,0.]) # to be modified
P0 = 1000*np.eye(5)
xhatP0 = np.hstack([xhat0, P0[0, 0], P0[0, 1], P0[0, 2], P0[0, 3], P0[0, 4], P0[1, 1],
                   P0[1, 2], P0[1, 3], P0[1, 4], P0[2, 2], P0[2, 3], P0[2, 4], P0[3, 3], P0[3, 4], P0[4, 4]])
# stack inline xhat0 and P0
# print("p0", xhatP0)


def jac_f_ext(t, x):
    # jacobian matrix of the dynamics f_ext
    # t does not seem to be used
    return np.array([
        [0., 0., -x[3]*np.sin(x[2]), np.cos(x[2]), 0.],
        [0., 0., x[3]*np.cos(x[2]), np.sin(x[2]), 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]
    ])


# print(jac_f_ext(0, xhat0))


def jac_h_ext(t, x):
    # jacobian matrix of the output map h_ext = (x1,x2)
    return np.array([
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.]
    ])


# print(jac_h_ext(0, xhat0))


def f_EKF(t, xhatP):
    """inputs : t, xhat(t) and P(t), outputs : xhat_dot(t) and P_dot(t)"""
    xhat = xhatP[:5]  # h_ext(x) = xhat[:2]
    Ps = xhatP[5:]
    P = np.array([[Ps[0], Ps[1], Ps[2], Ps[3], Ps[4]],
                  [Ps[1], Ps[5], Ps[6], Ps[7], Ps[8]],
                  [Ps[2], Ps[6], Ps[9], Ps[10], Ps[11]],
                  [Ps[3], Ps[7], Ps[10], Ps[12], Ps[13]],
                  [Ps[4], Ps[8], Ps[11], Ps[13], Ps[14]]])
    jac_f = jac_f_ext(t, xhat)
    jac_h = jac_h_ext(t, xhat)
    
    # enlever le facteur zero pour ajouter le bruit
    y_mes = y_traj(t) + np.random.normal(0.0, std_y)
    
    prod1 = mat_mul([P, np.transpose(jac_h), Rinv, y_mes - xhat[:2]])
    # print("prod1",prod1,type(prod1))
    
    xhat_dot =  f_ext(t,xhat) + prod1
    # print("xhat_dot",xhat_dot)
    
    prod2 = mat_mul([P, np.transpose(jac_h), Rinv, jac_h, P])
    # print("prod2",prod2,type(prod2))
    P_dot = lbda*P + np.dot(jac_f, P) + np.dot(P, np.transpose(jac_f)) + Q - prod2
    
    # return xhat_dot, P_dot[0,0]
    return np.hstack([xhat_dot, P_dot[0, 0], P_dot[0, 1], P_dot[0, 2], P_dot[0, 3], P_dot[0, 4], P_dot[1, 1], P_dot[1, 2], P_dot[1, 3], P_dot[1, 4], P_dot[2, 2], P_dot[2, 3], P_dot[2, 4], P_dot[3, 3], P_dot[3, 4], P_dot[4, 4]])


# print(f_EKF(10, xhatP0))

t_list, xhatP_list = solve_euler_explicit(f_EKF, xhatP0, .001, t0, tf)
xhat_list = xhatP_list[:,:5] # to be sliced, we need the first 5 coords of each element
plt.figure()
plt.plot(xhat_list[:,0],xhat_list[:,1])
plt.grid(True)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')


<IPython.core.display.Javascript object>

Text(0, 0.5, '$x_2$')

**Variation de $P_0$:** n'entraîne aucune différence notable sur l'estimation, si ce n'est une divergence pour une norme trop grande.

In [26]:
# scenario de mesure
std_y = 1.0  # ecart-type du bruit de mesure

# parametres du filtre de Kalman
# n = 5, r = 2

R =  np.eye(2)
Rinv = np.linalg.inv(R)
Q = np.eye(5)
lbda = 1
xhat0 = np.array([20.,0.,0.,0.,0.]) # to be modified
P0 = np.eye(5)
xhatP0 = np.hstack([xhat0, P0[0, 0], P0[0, 1], P0[0, 2], P0[0, 3], P0[0, 4], P0[1, 1],
                   P0[1, 2], P0[1, 3], P0[1, 4], P0[2, 2], P0[2, 3], P0[2, 4], P0[3, 3], P0[3, 4], P0[4, 4]])
# stack inline xhat0 and P0
# print("p0", xhatP0)


def jac_f_ext(t, x):
    # jacobian matrix of the dynamics f_ext
    # t does not seem to be used
    return np.array([
        [0., 0., -x[3]*np.sin(x[2]), np.cos(x[2]), 0.],
        [0., 0., x[3]*np.cos(x[2]), np.sin(x[2]), 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]
    ])


# print(jac_f_ext(0, xhat0))


def jac_h_ext(t, x):
    # jacobian matrix of the output map h_ext = (x1,x2)
    return np.array([
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.]
    ])


# print(jac_h_ext(0, xhat0))


def f_EKF(t, xhatP):
    """inputs : t, xhat(t) and P(t), outputs : xhat_dot(t) and P_dot(t)"""
    xhat = xhatP[:5]  # h_ext(x) = xhat[:2]
    Ps = xhatP[5:]
    P = np.array([[Ps[0], Ps[1], Ps[2], Ps[3], Ps[4]],
                  [Ps[1], Ps[5], Ps[6], Ps[7], Ps[8]],
                  [Ps[2], Ps[6], Ps[9], Ps[10], Ps[11]],
                  [Ps[3], Ps[7], Ps[10], Ps[12], Ps[13]],
                  [Ps[4], Ps[8], Ps[11], Ps[13], Ps[14]]])
    jac_f = jac_f_ext(t, xhat)
    jac_h = jac_h_ext(t, xhat)
    
    # enlever le facteur zero pour ajouter le bruit
    y_mes = y_traj(t) + np.random.normal(0.0, std_y)
    
    prod1 = mat_mul([P, np.transpose(jac_h), Rinv, y_mes - xhat[:2]])
    # print("prod1",prod1,type(prod1))
    
    xhat_dot =  f_ext(t,xhat) + prod1
    # print("xhat_dot",xhat_dot)
    
    prod2 = mat_mul([P, np.transpose(jac_h), Rinv, jac_h, P])
    # print("prod2",prod2,type(prod2))
    P_dot = lbda*P + np.dot(jac_f, P) + np.dot(P, np.transpose(jac_f)) + Q - prod2
    
    # return xhat_dot, P_dot[0,0]
    return np.hstack([xhat_dot, P_dot[0, 0], P_dot[0, 1], P_dot[0, 2], P_dot[0, 3], P_dot[0, 4], P_dot[1, 1], P_dot[1, 2], P_dot[1, 3], P_dot[1, 4], P_dot[2, 2], P_dot[2, 3], P_dot[2, 4], P_dot[3, 3], P_dot[3, 4], P_dot[4, 4]])


# print(f_EKF(10, xhatP0))

t_list, xhatP_list = solve_euler_explicit(f_EKF, xhatP0, .001, t0, tf)
xhat_list = xhatP_list[:,:5] # to be sliced, we need the first 5 coords of each element
plt.figure()
plt.plot(xhat_list[:,0],xhat_list[:,1])
plt.grid(True)
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')


<IPython.core.display.Javascript object>

Text(0, 0.5, '$x_2$')

**Analyse:** changer $x_0$ dans le filtre sans le changer dans la trajectoire réelle, augmente de façon évidente le bruit du modèle aux temps courts

Le filtre de Kalman etendu continu et deterministe presente ici a plusieurs defauts :
- la convergence n'est que locale (mais cela vient de la nonlinearite et on n'y echappe pas, à moins de recourir à des algorithmes tres differents et plus complexes),
- il necessite d'avoir à disposition la mesure GPS en tout temps,
- il ne dit pas comment choisir les parametres.

En pratique, on implemente plutôt la discretisation du filtre de Kalman, qui permet de predire par le modele (Euler explicite) la position du bateau lorsqu'aucune mesure n'est disponible, puis de "recâler/corriger" l'estimee lorsque la mesure est disponible, en comparant la mesure à l'estimee comme ci-dessus. 

Par ailleurs, le filtre de Kalman admet, dans le contexte lineaire, une interpretation stochastique qui permet de le voir comme le filtre optimal qui minimise la variance de l'erreur d'estimation lorsque les incertitudes de mesures et de modeles sont interpretees comme des bruits blancs gaussiens. C'est en fait ainsi qu'il a ete invente. Les matrices $R$ et $Q$ sont alors directement relies à la variance des incertitudes, ce qui en fait un filtre tres intuitif à regler et ce qui justifie son succes dans l'industrie. Vous verrez donc la suite de ce projet en cours de Probabilites !